In [1]:
import sys
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Desktop\\endtoend-mlops\\projects\\covid_chest_classifier\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Desktop\\endtoend-mlops\\projects\\covid_chest_classifier'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.ccclassifier.constants import *
from src.ccclassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import zipfile
import gdown
from src.ccclassifier import logger
from src.ccclassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-31 16:29:55,459: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-31 16:29:55,462: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-31 16:29:55,463: INFO: common: created directory at: artifacts]
[2025-08-31 16:29:55,465: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-31 16:29:55,466: INFO: 3390099456: Downloading data from https://drive.google.com/file/d/18OkRLS8T1gLfFAilJJF0alWBYx6NysEA/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?id=18OkRLS8T1gLfFAilJJF0alWBYx6NysEA
From (redirected): https://drive.google.com/uc?id=18OkRLS8T1gLfFAilJJF0alWBYx6NysEA&confirm=t&uuid=c5b64a91-7f7e-42a9-8884-e5a3238f3b1b
To: c:\Users\DELL\Desktop\endtoend-mlops\projects\covid_chest_classifier\artifacts\data_ingestion\data.zip
100%|██████████| 165M/165M [17:13<00:00, 160kB/s]    

[2025-08-31 16:47:13,250: INFO: 3390099456: Downloaded data from https://drive.google.com/file/d/18OkRLS8T1gLfFAilJJF0alWBYx6NysEA/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
